In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils
from holodeck.constants import MSOL, PC, YR, MPC, GYR, NWTG

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
m1 = 1e8 * MSOL
m2 = 1e9 * MSOL

gbh = holo.observations.Kormendy_Ho_2013()
COUL = 10.0

mt = m1 + m2
mbulge = gbh.mbulge_from_mbh(mt)
print(f"{mbulge/MSOL=:.2e}")
vdisp = gbh.vdisp_from_mbh(mt)
print(f"{vdisp/1e5=:.2e}")

mdf = mbulge

rchar = holo.evolution._radius_stellar_characteristic_dabringhausen_2008(mbulge)
print(f"{rchar/PC=:.2e}")
dens = holo.evolution._density_at_influence_radius_dehnen(mt, mbulge)
print(f"{dens*PC**3/MSOL=:.4e}")

rho = mbulge / (4.0*np.pi*rchar**3/3.0)
print(f"{rho*PC**3/MSOL=:.4e}")

tdyn = 1.0 / np.sqrt(NWTG*dens)
print(f"{tdyn/GYR*1e3=:.4e}")

dvdt = 2*np.pi*NWTG**2 * mdf * dens * COUL / vdisp**2
print(f"{dvdt*YR/1e5=:.4e}")

dadt = 2 * dvdt * tdyn
print(f"{dadt*YR/PC=:.4e}")

thard = rchar / dadt
print(f"{thard/GYR=:.4e}")

In [ ]:
sam = holo.sam.Semi_Analytic_Model()

In [ ]:
pop = holo.population.Pop_SAM(sam)

In [ ]:
# ---- Create initial population

pop = holo.population.Pop_Illustris()

# ---- Apply population modifiers

# resample to increase the number of binaries
mod_resamp = holo.PM_Resample(resample=5.0)
# modify population (in-place)
pop.modify(mod_resamp)


In [ ]:
pop.mass